Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (18724, 28, 28, 1) (18724, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size / 4 * image_size / 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 3.37652
Minibatch accuracy: 6.2%
Validation accuracy: 10.1%
Minibatch loss at step 50 : 2.23677
Minibatch accuracy: 12.5%
Validation accuracy: 31.6%
Minibatch loss at step 100 : 1.2657
Minibatch accuracy: 50.0%
Validation accuracy: 65.0%
Minibatch loss at step 150 : 1.11552
Minibatch accuracy: 56.2%
Validation accuracy: 74.0%
Minibatch loss at step 200 : 0.496026
Minibatch accuracy: 93.8%
Validation accuracy: 76.1%
Minibatch loss at step 250 : 0.694973
Minibatch accuracy: 81.2%
Validation accuracy: 74.4%
Minibatch loss at step 300 : 0.979662
Minibatch accuracy: 62.5%
Validation accuracy: 78.5%
Minibatch loss at step 350 : 0.336353
Minibatch accuracy: 93.8%
Validation accuracy: 79.2%
Minibatch loss at step 400 : 0.405566
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
Minibatch loss at step 450 : 0.147455
Minibatch accuracy: 93.8%
Validation accuracy: 78.9%
Minibatch loss at step 500 : 0.619199
Minibatch accuracy: 75.0%
Validation accuracy: 81

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size / 4 * image_size / 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    print hidden1.get_shape()
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                         padding='SAME', name='pool1')
    print pool1.get_shape()
    conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_biases)
    print hidden2.get_shape()
    pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                         padding='SAME', name='pool2')
    shape = pool2.get_shape().as_list()
    print pool2.get_shape()
    reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden3, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

TensorShape([Dimension(16), Dimension(28), Dimension(28), Dimension(16)])
TensorShape([Dimension(16), Dimension(14), Dimension(14), Dimension(16)])
TensorShape([Dimension(16), Dimension(14), Dimension(14), Dimension(16)])
TensorShape([Dimension(16), Dimension(7), Dimension(7), Dimension(16)])
TensorShape([Dimension(10000), Dimension(28), Dimension(28), Dimension(16)])
TensorShape([Dimension(10000), Dimension(14), Dimension(14), Dimension(16)])
TensorShape([Dimension(10000), Dimension(14), Dimension(14), Dimension(16)])
TensorShape([Dimension(10000), Dimension(7), Dimension(7), Dimension(16)])
TensorShape([Dimension(18724), Dimension(28), Dimension(28), Dimension(16)])
TensorShape([Dimension(18724), Dimension(14), Dimension(14), Dimension(16)])
TensorShape([Dimension(18724), Dimension(14), Dimension(14), Dimension(16)])
TensorShape([Dimension(18724), Dimension(7), Dimension(7), Dimension(16)])


In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 2.95208
Minibatch accuracy: 12.5%
Validation accuracy: 17.0%
Minibatch loss at step 50 : 0.699944
Minibatch accuracy: 75.0%
Validation accuracy: 72.3%
Minibatch loss at step 100 : 0.730935
Minibatch accuracy: 75.0%
Validation accuracy: 79.5%
Minibatch loss at step 150 : 0.892793
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Minibatch loss at step 200 : 0.210871
Minibatch accuracy: 93.8%
Validation accuracy: 80.3%
Minibatch loss at step 250 : 0.707908
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
Minibatch loss at step 300 : 0.886783
Minibatch accuracy: 75.0%
Validation accuracy: 81.6%
Minibatch loss at step 350 : 0.303945
Minibatch accuracy: 93.8%
Validation accuracy: 81.1%
Minibatch loss at step 400 : 0.212461
Minibatch accuracy: 100.0%
Validation accuracy: 82.6%
Minibatch loss at step 450 : 0.109636
Minibatch accuracy: 100.0%
Validation accuracy: 81.9%
Minibatch loss at step 500 : 0.710915
Minibatch accuracy: 75.0%
Validation accur

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [38]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
keep_prob = 0.5
wd = 0.004
NUM_EPOCHS_PER_DECAY = 1
LEARNING_RATE_DECAY_FACTOR = 0.2
initial_learning_rate = 0.01
graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    conv1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.01))
    hidden1_biases = tf.Variable(tf.constant(0.1, shape=[depth]))
    conv2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.01))
    hidden2_biases = tf.Variable(tf.constant(0.1, shape=[depth]))
    
    fc1_weights = tf.Variable(tf.truncated_normal(
      [image_size / 4 * image_size / 4 * depth, num_hidden], stddev=0.5))
    fc1_weights_decay = tf.mul(tf.nn.l2_loss(fc1_weights), wd, name="weight_loss")
    tf.add_to_collection('losses', fc1_weights_decay)

    fc1_biases = tf.Variable(tf.constant(0.1, shape=[num_hidden]))
    
    fc2_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.5))
    fc2_weights_decay = tf.mul(tf.nn.l2_loss(fc2_weights), wd, name="weight_loss")
    tf.add_to_collection('losses', fc2_weights_decay)
    
    fc2_biases = tf.Variable(tf.constant(0.1, shape=[num_labels]))
  
    # Model.
    def model_train(data):
        conv1 = tf.nn.conv2d(data, conv1_weights, [1, 1, 1, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + hidden1_biases)
        pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                             padding='SAME', name='pool1')
        conv2 = tf.nn.conv2d(pool1, conv2_weights, [1, 1, 1, 1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + hidden2_biases)
        pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                             padding='SAME', name='pool2')
        shape = pool2.get_shape().as_list()
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        reshape_dropout = tf.nn.dropout(reshape, keep_prob)
        hidden3 = tf.nn.relu(tf.matmul(reshape_dropout, fc1_weights) + fc1_biases)
        hidden_dropout = tf.nn.dropout(hidden3, keep_prob)
        return tf.matmul(hidden_dropout, fc2_weights) + fc2_biases

    def model_test(data):
        conv1 = tf.nn.conv2d(data, conv1_weights, [1, 1, 1, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + hidden1_biases)
        pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                             padding='SAME', name='pool1')
        conv2 = tf.nn.conv2d(pool1, conv2_weights, [1, 1, 1, 1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + hidden2_biases)
        pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                             padding='SAME', name='pool2')
        shape = pool2.get_shape().as_list()
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden3 = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
        return tf.matmul(hidden3, fc2_weights) + fc2_biases
    
    # Training computation.
    logits = model_train(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    tf.add_to_collection('losses', loss)

    total_loss = tf.add_n(tf.get_collection('losses'), name='total_loss')

    global_step = tf.Variable(0)  # count the number of steps taken.
    nr_batches_per_epoch = train_labels.shape[0] / batch_size
    decay_steps = int(nr_batches_per_epoch * NUM_EPOCHS_PER_DECAY)
    print decay_steps
    
    learning_rate = tf.train.exponential_decay(
        initial_learning_rate,
        global_step,
        decay_steps,
        LEARNING_RATE_DECAY_FACTOR,
        staircase=True)
    
    optimizer = tf.train.GradientDescentOptimizer(initial_learning_rate).minimize(total_loss, global_step=global_step)
    # Optimizer.
#     optimizer = tf.train.GradientDescentOptimizer(0.004).minimize(total_loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model_test(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model_test(tf_test_dataset))

12500


In [39]:
num_steps = 3000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print "Initialized"
    for step in xrange(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print "Minibatch loss at step", step, ":", l
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            print "Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 5.66488
Minibatch accuracy: 18.8%
Validation accuracy: 8.7%


InvalidArgumentError: ReluGrad input is not finite. : Tensor had Inf and NaN values
	 [[Node: gradients/Relu_2_grad/Relu_2/CheckNumerics = CheckNumerics[T=DT_FLOAT, message="ReluGrad input is not finite.", _device="/job:localhost/replica:0/task:0/cpu:0"](add_2)]]
Caused by op u'gradients/Relu_2_grad/Relu_2/CheckNumerics', defined at:
  File "/Users/ioanaveronicachelu/anaconda/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/ioanaveronicachelu/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/ioanaveronicachelu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/ioanaveronicachelu/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 589, in launch_instance
    app.start()
  File "/Users/ioanaveronicachelu/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 403, in